In [223]:
import pandas as pd
from os.path import exists
import http.client
import json
from cmapPy.pandasGEXpress.parse import parse
import requests
from io import StringIO
import os


In [5]:
# read cmap valid genes
cmap_genes_df = pd.read_csv('sigs_to_test/cmap_genes.txt', sep= '\t')
cmap_genes_df = cmap_genes_df[~(cmap_genes_df['Type'] =='not inferred')]
cmap_genes = set(cmap_genes_df['Entrez ID'].astype(str))
cmap_gene_names = set(cmap_genes_df['Symbol'].astype(str))

In [32]:
# read lincs pert names
lincs_drugs = set(pd.read_csv('lincs_drugs.csv')['drug'].str.lower())

# GSE60408 processing

In [300]:
GSE60408_perts = []
objs_GSE60408 = list(pd.read_csv("sigs_to_test/GSE60408_objects/rdata_objects.csv", header = None)[0])
# collect perts
for i in objs_GSE60408:
    GSE60408_perts.append(i.split('_')[0].lower())
for i,a in enumerate(GSE60408_perts):
    GSE60408_perts[i] = a.replace(".", "-")

# GSE102006 processing

In [75]:
GSE102006_perts = []
objs_GSE102006 = list(pd.read_csv("sigs_to_test/GSE102006_objects/rdata_objects.csv", header = None)[0])
# collect perts
for i in objs_GSE102006:
    GSE102006_perts.append(i.split('_')[2].lower())
for i,a in enumerate(GSE102006_perts):
    GSE102006_perts[i] = a.replace(".", "-")

# GSE51068 processing


In [269]:
GSE51068_perts = []
objs_GSE51068 = list(pd.read_csv("sigs_to_test/GSE51068_objects/rdata_objects.csv", header = None)[0])
# collect perts
for i in objs_GSE51068:
    GSE51068_perts.append(i.split('_')[0].lower())
for i,a in enumerate(GSE51068_perts):
    GSE51068_perts[i] = a.replace(".", "-")

# CLUE

In [ ]:
dataset = "GSE51068"
objs = pd.read_csv(f"sigs_to_test/{dataset}_objects/rdata_objects.csv", header = None)
jobs = {}
conn = http.client.HTTPSConnection("api.clue.io")

headers = {
  'Content-Type': 'application/json',
  'Accept' : 'application/json',
  'user_key' : ''
}
sig_len = {}
index = 0
for obj in list(set(objs[0])):
    pert_name = obj.split('_')[0]
    name = f"sigs_to_test/{dataset}_objects_results/{obj}.csv"
    file_exists = exists(name)
    if not file_exists:
        continue
        
    df = pd.read_csv(name, sep = '\t', dtype={'gene_id': str})
    df = df[df['gene_id'].notna()]
    df = df[~df['gene_id'].astype(str).str.contains("///")]
    df = df[~df['gene_id'].astype(str).str.contains("---")]
    df = df[~df['gene_id'].astype(str).str.contains("NA")]
    up_genes = set(df[df['logFC']>0]['gene_id']) & cmap_genes    
    down_genes = set(df[df['logFC']<0]['gene_id']) & cmap_genes
    genes_to_remove = up_genes & down_genes
    
    for gene in genes_to_remove:
        up_genes.remove(gene)
        down_genes.remove(gene)
        
    up_genes = list(up_genes)
    down_genes = list(down_genes)
    
    genes_column = up_genes + down_genes
    
    sig_len[obj] = (len(up_genes), len(down_genes))
    if(len(up_genes) < 10 or len(up_genes) > 150):
        continue
    if(len(down_genes) < 10 or len(down_genes) > 150):
        continue
    up_genes_str = '\t'.join(list(up_genes))
    down_genes_str ='\t'.join(list(down_genes))
    dump = { "tool_id": "sig_gutc_tool",
            "data_type": "L1000",
            "name": f"{obj}",
            "uptag-cmapfile": "TAG\t"+up_genes_str,
            "dntag-cmapfile" :"TAG\t"+down_genes_str,
            "dataset": "Touchstone" 
           }
    payload =  json.dumps(dump)
    print(obj)
    conn.request("POST", "/api/jobs", payload, headers)
    res = conn.getresponse()
    data = res.read()
    print(data.decode("utf-8"))
    j = json.loads(data)
    if 'result' in j:
        jobs[obj] = j['result']['job_id']
    if len(jobs) == 10:
        break


# iLINCS
Save  data for iLINCS

In [308]:
dataset = "GSE60408"
objs = pd.read_csv(f"sigs_to_test/{dataset}_objects/rdata_objects.csv", header = None)

for obj in list(set(objs[0])):
    pert_name = obj.split('_')[0]
    name = f"sigs_to_test/{dataset}_objects_results/{obj}.csv"
    file_exists = exists(name)
    if not file_exists:
        continue
    df = pd.read_csv(name, sep = '\t', dtype={'gene_id': str})
    df = df[df['gene_id'].notna()]
    df = df[~df['gene_id'].astype(str).str.contains("///")]
    df = df[~df['gene_id'].astype(str).str.contains("---")]
    df = df[~df['gene_id'].astype(str).str.contains("NA")]
    up_genes = set(df[df['logFC']>0]['gene_id'])    
    down_genes = set(df[df['logFC']<0]['gene_id'])
    genes_to_remove = up_genes & down_genes
    for gene in genes_to_remove:
        up_genes.remove(gene)
        down_genes.remove(gene)
    up_genes = list(up_genes)
    down_genes = list(down_genes)
    if df.empty:
        continue
        
    df_dict = dict(zip(list(df['gene_id']),list(df['gene_name'])))
    df_dict_df = pd.DataFrame(df_dict.values(), df_dict.keys())
    up_gene_names = list(df_dict_df.loc[list(up_genes),][0])
    down_gene_names = list(df_dict_df.loc[list(down_genes),][0])
    genes_column = up_genes + down_genes
    genes_column_names = up_gene_names + down_gene_names
    logfc_column = [1]*len(up_genes) + [-1]*len(down_genes) 
    ilincs_df = pd.DataFrame({'Name_GeneSymbol':genes_column, 'Value_LogDiffExp':logfc_column, 'Name': genes_column_names})
    ilincs_df.to_csv(f'sigs_to_test\\{dataset}_ilincs\\{obj}_ilincs.csv',sep = '\t', index=False)

In [33]:
objs_GSE102006 = list(pd.read_csv("sigs_to_test/GSE102006_objects/rdata_objects.csv", header = None)[0])
objs_GSE51068 = list(pd.read_csv("sigs_to_test/GSE51068_objects/rdata_objects.csv", header = None)[0])
objs_GSE60408 = list(pd.read_csv("sigs_to_test/GSE60408_objects/rdata_objects.csv", header = None)[0])
GSE51068_perts = []
for i in objs_GSE51068:
    GSE51068_perts.append(i.split('_')[0].lower())
for i,a in enumerate(GSE51068_perts):
    if a == 'trichostatina':
        GSE51068_perts[i]='trichostatin-a'
    if a == 'aclacinomycina':
        GSE51068_perts[i]='aclacinomycin-a'
    if a == 'h-7_dihydrochloride':
        GSE51068_perts[i]='h-7'
    if a == 'mitomycinc':
        GSE51068_perts[i]='mitomycin-c'

GSE102006_perts = []
for i in objs_GSE102006:
    GSE102006_perts.append(i.split('_')[2].lower())

# L1000FWD

In [115]:
def get_signature(up_genes, down_genes):
    L1000FWD_URL = "https://maayanlab.cloud/L1000FWD/"
    payload = {
        'up_genes': up_genes,
        'down_genes': down_genes
    }

    response = requests.post(L1000FWD_URL + 'sig_search', json=payload)
    df = None
    if response.status_code == 200:
        sig_id = response.json()['result_id']

        download_url = f'https://maayanlab.cloud/L1000FWD/result/download/{sig_id}'
        print(download_url)

        response2 = requests.get(download_url)

        print(response2.status_code)

        response_text = response2.text
        string_io = StringIO(response_text)

        df = pd.read_csv(string_io)
    return df


In [116]:
dataset = "GSE51068"
L1000FWD_URL = "https://maayanlab.cloud/L1000FWD/"
for obj in list(set(objs[0])):
    pert_name = obj.split('_')[0]
    name = f"sigs_to_test/{dataset}_objects_results/{obj}.csv"
    file_exists = exists(name)
    if not file_exists:
        continue
    df = pd.read_csv(name, sep = '\t', dtype={'gene_id': str})
    df = df[df['gene_name'].notna()]
    df = df[~df['gene_name'].astype(str).str.contains("///")]
    df = df[~df['gene_name'].astype(str).str.contains("---")]
    df = df[~df['gene_name'].astype(str).str.contains("NA")]
    up_genes = set(df[df['logFC']>0]['gene_name'])   
    down_genes = set(df[df['logFC']<0]['gene_name'])
    genes_to_remove = up_genes & down_genes
    for gene in genes_to_remove:
        up_genes.remove(gene)
        down_genes.remove(gene)
    up_genes = list(up_genes)
    down_genes = list(down_genes)
    
    payload = {'up_genes': up_genes,
           'down_genes': down_genes
    }
    df = get_signature(up_genes, down_genes)
    if df is None:
        continue
        
    df.to_csv(f'sigs_to_test//{dataset}_L1000FWD//{obj}_L1000FWD.csv', sep = '\t')

https://maayanlab.cloud/L1000FWD/result/download/62c44d67c468ed002cd44694
200
https://maayanlab.cloud/L1000FWD/result/download/62c44d82c468ed002cd44695
200
https://maayanlab.cloud/L1000FWD/result/download/62c44daac468ed002cd44696
200
https://maayanlab.cloud/L1000FWD/result/download/62c44dcec468ed002cd44697
200
https://maayanlab.cloud/L1000FWD/result/download/62c44df1c468ed002cd44698
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e16c468ed002cd44699
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e31c468ed002cd4469a
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e42c468ed002cd4469b
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e55c468ed002cd4469c
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e67c468ed002cd4469d
200
https://maayanlab.cloud/L1000FWD/result/download/62c44e89c468ed002cd4469e
200
https://maayanlab.cloud/L1000FWD/result/download/62c44ea9c468ed002cd4469f
200
https://maayanlab.cloud/L1000FWD/result/download/62c44ed9c468ed0

In [ ]:
    response = requests.post(L1000FWD_URL + 'sig_search', json=payload)
    if response.status_code == 200:
        result_id = str(response.json()).split("'")[3]
    else:
        print("1")
        print(obj)
        break
    response = requests.get(L1000FWD_URL + 'result/topn/' + result_id)
    if response.status_code != 200:
        print("2")
        print(obj)
        break
    sigs_df = pd.DataFrame(columns = ['qvals','combined_scores','pvals','sig_id','scores','zscores','pert_id','pert_iname'])
    for sig in response.json()['similar']:
        s = sig['sig_id'].split(':')[1]
        pert_id = s.split('-')[0]+"-"+s.split('-')[1]
        pert_iname = inst_info[inst_info['pert_id'] == pert_id]['pert_iname'][0]
        sigs_df.loc[sigs_df.shape[0]-1]= [sig['qvals'], sig['combined_scores'], sig['pvals'], sig['sig_id'], sig['scores'], sig['zscores'], pert_id,pert_iname]
    sigs_df.to_csv(f'sigs_to_test//{dataset}_L1000FWD//{obj}_L1000FWD.csv', sep = '\t')

# Checking iLincs ranks

In [373]:
# GSE60408
dataset_file_pert = {"NITROFURAZONE_timept12hrs_rep1.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept12hrs_rep2.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept12hrs_rep3.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept24hrs_rep1.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept24hrs_rep2.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept24hrs_rep3.Rdata" : "nitrofurazone",
"ACETOHEXAMIDE_timept6hrs_rep1.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept6hrs_rep2.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept6hrs_rep3.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept12hrs_rep1.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept12hrs_rep2.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept12hrs_rep3.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept24hrs_rep1.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept24hrs_rep2.Rdata" : "acetohexamide",
"ACETOHEXAMIDE_timept24hrs_rep3.Rdata" : "acetohexamide",
"ACRISORCIN_timept6hrs_rep1.Rdata" : "acrisorcin",
"ACRISORCIN_timept6hrs_rep2.Rdata" : "acrisorcin",
"ACRISORCIN_timept6hrs_rep3.Rdata" : "acrisorcin",
"ACRISORCIN_timept12hrs_rep1.Rdata" : "acrisorcin",
"ACRISORCIN_timept12hrs_rep2.Rdata" : "acrisorcin",
"ACRISORCIN_timept12hrs_rep3.Rdata" : "acrisorcin",
"ACRISORCIN_timept24hrs_rep1.Rdata" : "acrisorcin",
"ACRISORCIN_timept24hrs_rep2.Rdata" : "acrisorcin",
"ACRISORCIN_timept24hrs_rep3.Rdata" : "acrisorcin",
"ALPROSTADIL_timept6hrs_rep1.Rdata" : "alprostadil",
"ALPROSTADIL_timept6hrs_rep2.Rdata" : "alprostadil",
"ALPROSTADIL_timept6hrs_rep3.Rdata" : "alprostadil",
"ALPROSTADIL_timept12hrs_rep1.Rdata" : "alprostadil",
"ALPROSTADIL_timept12hrs_rep2.Rdata" : "alprostadil",
"ALPROSTADIL_timept12hrs_rep3.Rdata" : "alprostadil",
"ALPROSTADIL_timept24hrs_rep1.Rdata" : "alprostadil",
"ALPROSTADIL_timept24hrs_rep2.Rdata" : "alprostadil",
"ALPROSTADIL_timept24hrs_rep3.Rdata" : "alprostadil",
"ALTRETAMINE_timept6hrs_rep1.Rdata" : "altretamine",
"ALTRETAMINE_timept6hrs_rep2.Rdata" : "altretamine",
"ALTRETAMINE_timept6hrs_rep3.Rdata" : "altretamine",
"ALTRETAMINE_timept12hrs_rep1.Rdata" : "altretamine",
"ALTRETAMINE_timept12hrs_rep2.Rdata" : "altretamine",
"ALTRETAMINE_timept12hrs_rep3.Rdata" : "altretamine",
"ALTRETAMINE_timept24hrs_rep1.Rdata" : "altretamine",
"ALTRETAMINE_timept24hrs_rep2.Rdata" : "altretamine",
"ALTRETAMINE_timept24hrs_rep3.Rdata" : "altretamine",
"AMIODARONE.HYDROCHLORIDE_timept6hrs_rep1.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept6hrs_rep2.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept6hrs_rep3.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept12hrs_rep1.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept12hrs_rep2.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept12hrs_rep3.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept24hrs_rep1.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept24hrs_rep2.Rdata" : "amiodarone-hydrochloride",
"AMIODARONE.HYDROCHLORIDE_timept24hrs_rep3.Rdata" : "amiodarone-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept6hrs_rep1.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept6hrs_rep2.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept6hrs_rep3.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept12hrs_rep1.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept12hrs_rep2.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept12hrs_rep3.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept24hrs_rep1.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept24hrs_rep2.Rdata" : "apomorphine-hydrochloride",
"APOMORPHINE.HYDROCHLORIDE_timept24hrs_rep3.Rdata" : "apomorphine-hydrochloride",
"Auranofin_timept6hrs_rep1.Rdata" : "auranofin",
"Auranofin_timept6hrs_rep2.Rdata" : "auranofin",
"Auranofin_timept6hrs_rep3.Rdata" : "auranofin",
"Auranofin_timept12hrs_rep1.Rdata" : "auranofin",
"Auranofin_timept12hrs_rep2.Rdata" : "auranofin",
"Auranofin_timept12hrs_rep3.Rdata" : "auranofin",
"Auranofin_timept24hrs_rep1.Rdata" : "auranofin",
"Auranofin_timept24hrs_rep2.Rdata" : "auranofin",
"Auranofin_timept24hrs_rep3.Rdata" : "auranofin",
"AZATHIOPRINE_timept6hrs_rep1.Rdata" : "azathioprine",
"AZATHIOPRINE_timept6hrs_rep2.Rdata" : "azathioprine",
"AZATHIOPRINE_timept6hrs_rep3.Rdata" : "azathioprine",
"AZATHIOPRINE_timept12hrs_rep1.Rdata" : "azathioprine",
"AZATHIOPRINE_timept12hrs_rep2.Rdata" : "azathioprine",
"AZATHIOPRINE_timept12hrs_rep3.Rdata" : "azathioprine",
"AZATHIOPRINE_timept24hrs_rep1.Rdata" : "azathioprine",
"AZATHIOPRINE_timept24hrs_rep2.Rdata" : "azathioprine",
"AZATHIOPRINE_timept24hrs_rep3.Rdata" : "azathioprine",
"BRINZOLAMIDE_timept6hrs_rep1.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept6hrs_rep2.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept6hrs_rep3.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept12hrs_rep1.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept12hrs_rep2.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept12hrs_rep3.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept24hrs_rep1.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept24hrs_rep2.Rdata" : "brinzolamide",
"BRINZOLAMIDE_timept24hrs_rep3.Rdata" : "brinzolamide",
"Butoconazole.nitrate_timept6hrs_rep1.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept6hrs_rep2.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept6hrs_rep3.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept12hrs_rep1.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept12hrs_rep2.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept12hrs_rep3.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept24hrs_rep1.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept24hrs_rep2.Rdata" : "butoconazole-nitrate",
"Butoconazole.nitrate_timept24hrs_rep3.Rdata" : "butoconazole-nitrate",
"CEFIXIME_timept6hrs_rep1.Rdata" : "cefixime",
"CEFIXIME_timept6hrs_rep2.Rdata" : "cefixime",
"CEFIXIME_timept6hrs_rep3.Rdata" : "cefixime",
"CEFIXIME_timept12hrs_rep1.Rdata" : "cefixime",
"CEFIXIME_timept12hrs_rep2.Rdata" : "cefixime",
"CEFIXIME_timept12hrs_rep3.Rdata" : "cefixime",
"CEFIXIME_timept24hrs_rep1.Rdata" : "cefixime",
"CEFIXIME_timept24hrs_rep2.Rdata" : "cefixime",
"CEFIXIME_timept24hrs_rep3.Rdata" : "cefixime",
"CHLORMEZANONE_timept6hrs_rep1.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept6hrs_rep2.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept6hrs_rep3.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept12hrs_rep1.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept12hrs_rep2.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept12hrs_rep3.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept24hrs_rep1.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept24hrs_rep2.Rdata" : "chlormezanone",
"CHLORMEZANONE_timept24hrs_rep3.Rdata" : "chlormezanone",
"CHLOROXINE_timept6hrs_rep1.Rdata" : "chloroxine",
"CHLOROXINE_timept6hrs_rep2.Rdata" : "chloroxine",
"CHLOROXINE_timept6hrs_rep3.Rdata" : "chloroxine",
"CHLOROXINE_timept12hrs_rep1.Rdata" : "chloroxine",
"CHLOROXINE_timept12hrs_rep2.Rdata" : "chloroxine",
"CHLOROXINE_timept12hrs_rep3.Rdata" : "chloroxine",
"CHLOROXINE_timept24hrs_rep1.Rdata" : "chloroxine",
"CHLOROXINE_timept24hrs_rep2.Rdata" : "chloroxine",
"CHLOROXINE_timept24hrs_rep3.Rdata" : "chloroxine",
"Cladribine_timept6hrs_rep1.Rdata" : "cladribine",
"Cladribine_timept6hrs_rep2.Rdata" : "cladribine",
"Cladribine_timept6hrs_rep3.Rdata" : "cladribine",
"Cladribine_timept12hrs_rep1.Rdata" : "cladribine",
"Cladribine_timept12hrs_rep2.Rdata" : "cladribine",
"Cladribine_timept12hrs_rep3.Rdata" : "cladribine",
"Cladribine_timept24hrs_rep1.Rdata" : "cladribine",
"Cladribine_timept24hrs_rep2.Rdata" : "cladribine",
"Cladribine_timept24hrs_rep3.Rdata" : "cladribine",
"CLEMASTINE_timept6hrs_rep1.Rdata" : "clemastine",
"CLEMASTINE_timept6hrs_rep2.Rdata" : "clemastine",
"CLEMASTINE_timept6hrs_rep3.Rdata" : "clemastine",
"CLEMASTINE_timept12hrs_rep1.Rdata" : "clemastine",
"CLEMASTINE_timept12hrs_rep2.Rdata" : "clemastine",
"CLEMASTINE_timept12hrs_rep3.Rdata" : "clemastine",
"CLEMASTINE_timept24hrs_rep1.Rdata" : "clemastine",
"CLEMASTINE_timept24hrs_rep2.Rdata" : "clemastine",
"CLEMASTINE_timept24hrs_rep3.Rdata" : "clemastine",
"CLOMIPHENE.CITRATE_timept6hrs_rep1.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept6hrs_rep2.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept6hrs_rep3.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept12hrs_rep1.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept12hrs_rep2.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept12hrs_rep3.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept24hrs_rep1.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept24hrs_rep2.Rdata" : "clomiphene-citrate",
"CLOMIPHENE.CITRATE_timept24hrs_rep3.Rdata" : "clomiphene-citrate",
"CLOZAPINE_timept6hrs_rep1.Rdata" : "clozapine",
"CLOZAPINE_timept6hrs_rep2.Rdata" : "clozapine",
"CLOZAPINE_timept6hrs_rep3.Rdata" : "clozapine",
"CLOZAPINE_timept12hrs_rep1.Rdata" : "clozapine",
"CLOZAPINE_timept12hrs_rep2.Rdata" : "clozapine",
"CLOZAPINE_timept12hrs_rep3.Rdata" : "clozapine",
"CLOZAPINE_timept24hrs_rep1.Rdata" : "clozapine",
"CLOZAPINE_timept24hrs_rep2.Rdata" : "clozapine",
"CLOZAPINE_timept24hrs_rep3.Rdata" : "clozapine",
"CYTARABINE_timept6hrs_rep1.Rdata" : "cytarabine",
"CYTARABINE_timept6hrs_rep2.Rdata" : "cytarabine",
"CYTARABINE_timept6hrs_rep3.Rdata" : "cytarabine",
"CYTARABINE_timept12hrs_rep1.Rdata" : "cytarabine",
"CYTARABINE_timept12hrs_rep2.Rdata" : "cytarabine",
"CYTARABINE_timept12hrs_rep3.Rdata" : "cytarabine",
"CYTARABINE_timept24hrs_rep1.Rdata" : "cytarabine",
"CYTARABINE_timept24hrs_rep2.Rdata" : "cytarabine",
"CYTARABINE_timept24hrs_rep3.Rdata" : "cytarabine",
"DACTINOMYCIN_timept6hrs_rep1.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept6hrs_rep2.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept6hrs_rep3.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept12hrs_rep1.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept12hrs_rep2.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept12hrs_rep3.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept24hrs_rep1.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept24hrs_rep2.Rdata" : "dactinomycin",
"DACTINOMYCIN_timept24hrs_rep3.Rdata" : "dactinomycin",
"DAPSONE_timept6hrs_rep1.Rdata" : "dapsone",
"DAPSONE_timept6hrs_rep2.Rdata" : "dapsone",
"DAPSONE_timept6hrs_rep3.Rdata" : "dapsone",
"DAPSONE_timept12hrs_rep1.Rdata" : "dapsone",
"DAPSONE_timept12hrs_rep2.Rdata" : "dapsone",
"DAPSONE_timept12hrs_rep3.Rdata" : "dapsone",
"DAPSONE_timept24hrs_rep1.Rdata" : "dapsone",
"DAPSONE_timept24hrs_rep2.Rdata" : "dapsone",
"DAPSONE_timept24hrs_rep3.Rdata" : "dapsone",
"DEFEROXAMINE.MESYLATE_timept6hrs_rep1.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept6hrs_rep2.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept6hrs_rep3.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept12hrs_rep1.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept12hrs_rep2.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept12hrs_rep3.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept24hrs_rep1.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept24hrs_rep2.Rdata" : "deferoxamine-mesylate",
"DEFEROXAMINE.MESYLATE_timept24hrs_rep3.Rdata" : "deferoxamine-mesylate",
"DIGITOXIN_timept6hrs_rep1.Rdata" : "digitoxin",
"DIGITOXIN_timept6hrs_rep2.Rdata" : "digitoxin",
"DIGITOXIN_timept6hrs_rep3.Rdata" : "digitoxin",
"DIGITOXIN_timept12hrs_rep1.Rdata" : "digitoxin",
"DIGITOXIN_timept12hrs_rep2.Rdata" : "digitoxin",
"DIGITOXIN_timept12hrs_rep3.Rdata" : "digitoxin",
"DIGITOXIN_timept24hrs_rep1.Rdata" : "digitoxin",
"DIGITOXIN_timept24hrs_rep2.Rdata" : "digitoxin",
"DIGITOXIN_timept24hrs_rep3.Rdata" : "digitoxin",
"DIMERCAPROL_timept6hrs_rep1.Rdata" : "dimercaprol",
"DIMERCAPROL_timept6hrs_rep2.Rdata" : "dimercaprol",
"DIMERCAPROL_timept6hrs_rep3.Rdata" : "dimercaprol",
"DIMERCAPROL_timept12hrs_rep1.Rdata" : "dimercaprol",
"DIMERCAPROL_timept12hrs_rep2.Rdata" : "dimercaprol",
"DIMERCAPROL_timept12hrs_rep3.Rdata" : "dimercaprol",
"DIMERCAPROL_timept24hrs_rep1.Rdata" : "dimercaprol",
"DIMERCAPROL_timept24hrs_rep2.Rdata" : "dimercaprol",
"DIMERCAPROL_timept24hrs_rep3.Rdata" : "dimercaprol",
"Docetaxel_timept6hrs_rep1.Rdata" : "docetaxel",
"Docetaxel_timept6hrs_rep2.Rdata" : "docetaxel",
"Docetaxel_timept6hrs_rep3.Rdata" : "docetaxel",
"Docetaxel_timept12hrs_rep1.Rdata" : "docetaxel",
"Docetaxel_timept12hrs_rep2.Rdata" : "docetaxel",
"Docetaxel_timept12hrs_rep3.Rdata" : "docetaxel",
"Docetaxel_timept24hrs_rep1.Rdata" : "docetaxel",
"Docetaxel_timept24hrs_rep2.Rdata" : "docetaxel",
"Docetaxel_timept24hrs_rep3.Rdata" : "docetaxel",
"DYCLONINE.HYDROCHLORIDE_timept6hrs_rep1.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept6hrs_rep2.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept6hrs_rep3.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept12hrs_rep1.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept12hrs_rep2.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept12hrs_rep3.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept24hrs_rep1.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept24hrs_rep2.Rdata" : "dyclonine-hydrochloride",
"DYCLONINE.HYDROCHLORIDE_timept24hrs_rep3.Rdata" : "dyclonine-hydrochloride",
"ECONAZOLE.NITRATE_timept6hrs_rep1.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept6hrs_rep2.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept6hrs_rep3.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept12hrs_rep1.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept12hrs_rep2.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept12hrs_rep3.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept24hrs_rep1.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept24hrs_rep2.Rdata" : "econazole-nitrate",
"ECONAZOLE.NITRATE_timept24hrs_rep3.Rdata" : "econazole-nitrate",
"ENOXACIN_timept6hrs_rep1.Rdata" : "enoxacin",
"ENOXACIN_timept6hrs_rep2.Rdata" : "enoxacin",
"ENOXACIN_timept6hrs_rep3.Rdata" : "enoxacin",
"ENOXACIN_timept12hrs_rep1.Rdata" : "enoxacin",
"ENOXACIN_timept12hrs_rep2.Rdata" : "enoxacin",
"ENOXACIN_timept12hrs_rep3.Rdata" : "enoxacin",
"ENOXACIN_timept24hrs_rep1.Rdata" : "enoxacin",
"ENOXACIN_timept24hrs_rep2.Rdata" : "enoxacin",
"ENOXACIN_timept24hrs_rep3.Rdata" : "enoxacin",
"ERGOCALCIFEROL_timept6hrs_rep1.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept6hrs_rep2.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept6hrs_rep3.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept12hrs_rep1.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept12hrs_rep2.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept12hrs_rep3.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept24hrs_rep1.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept24hrs_rep2.Rdata" : "ergocalciferol",
"ERGOCALCIFEROL_timept24hrs_rep3.Rdata" : "ergocalciferol",
"ESTRADIOL_timept6hrs_rep1.Rdata" : "estradiol",
"ESTRADIOL_timept6hrs_rep2.Rdata" : "estradiol",
"ESTRADIOL_timept6hrs_rep3.Rdata" : "estradiol",
"ESTRADIOL_timept12hrs_rep1.Rdata" : "estradiol",
"ESTRADIOL_timept12hrs_rep2.Rdata" : "estradiol",
"ESTRADIOL_timept12hrs_rep3.Rdata" : "estradiol",
"ESTRADIOL_timept24hrs_rep1.Rdata" : "estradiol",
"ESTRADIOL_timept24hrs_rep2.Rdata" : "estradiol",
"ESTRADIOL_timept24hrs_rep3.Rdata" : "estradiol",
"ETHOPROPAZINE.HYDROCHLORIDE_timept6hrs_rep1.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept6hrs_rep2.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept6hrs_rep3.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept12hrs_rep1.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept12hrs_rep2.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept12hrs_rep3.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept24hrs_rep1.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept24hrs_rep2.Rdata" : "ethopropazine-hydrochloride",
"ETHOPROPAZINE.HYDROCHLORIDE_timept24hrs_rep3.Rdata" : "ethopropazine-hydrochloride",
"ETOPOSIDE_timept6hrs_rep1.Rdata" : "etoposide",
"ETOPOSIDE_timept6hrs_rep2.Rdata" : "etoposide",
"ETOPOSIDE_timept6hrs_rep3.Rdata" : "etoposide",
"ETOPOSIDE_timept12hrs_rep1.Rdata" : "etoposide",
"ETOPOSIDE_timept12hrs_rep2.Rdata" : "etoposide",
"ETOPOSIDE_timept12hrs_rep3.Rdata" : "etoposide",
"FELODIPINE_timept6hrs_rep1.Rdata" : "felodipine",
"FELODIPINE_timept6hrs_rep2.Rdata" : "felodipine",
"FELODIPINE_timept6hrs_rep3.Rdata" : "felodipine",
"FELODIPINE_timept12hrs_rep1.Rdata" : "felodipine",
"FELODIPINE_timept12hrs_rep2.Rdata" : "felodipine",
"FELODIPINE_timept12hrs_rep3.Rdata" : "felodipine",
"FELODIPINE_timept24hrs_rep1.Rdata" : "felodipine",
"FELODIPINE_timept24hrs_rep2.Rdata" : "felodipine",
"FELODIPINE_timept24hrs_rep3.Rdata" : "felodipine",
"FENOFIBRATE_timept6hrs_rep1.Rdata" : "fenofibrate",
"FENOFIBRATE_timept6hrs_rep2.Rdata" : "fenofibrate",
"FENOFIBRATE_timept6hrs_rep3.Rdata" : "fenofibrate",
"FENOFIBRATE_timept12hrs_rep1.Rdata" : "fenofibrate",
"FENOFIBRATE_timept12hrs_rep2.Rdata" : "fenofibrate",
"FENOFIBRATE_timept12hrs_rep3.Rdata" : "fenofibrate",
"FENOFIBRATE_timept24hrs_rep1.Rdata" : "fenofibrate",
"FENOFIBRATE_timept24hrs_rep2.Rdata" : "fenofibrate",
"FENOFIBRATE_timept24hrs_rep3.Rdata" : "fenofibrate",
"FLURBIPROFEN_timept6hrs_rep1.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept6hrs_rep2.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept6hrs_rep3.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept12hrs_rep1.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept12hrs_rep2.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept12hrs_rep3.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept24hrs_rep1.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept24hrs_rep2.Rdata" : "flurbiprofen",
"FLURBIPROFEN_timept24hrs_rep3.Rdata" : "flurbiprofen",
"FOLIC.ACID_timept6hrs_rep1.Rdata" : "folic-acid",
"FOLIC.ACID_timept6hrs_rep2.Rdata" : "folic-acid",
"FOLIC.ACID_timept6hrs_rep3.Rdata" : "folic-acid",
"FOLIC.ACID_timept12hrs_rep1.Rdata" : "folic-acid",
"FOLIC.ACID_timept12hrs_rep2.Rdata" : "folic-acid",
"FOLIC.ACID_timept12hrs_rep3.Rdata" : "folic-acid",
"FOLIC.ACID_timept24hrs_rep1.Rdata" : "folic-acid",
"FOLIC.ACID_timept24hrs_rep2.Rdata" : "folic-acid",
"FOLIC.ACID_timept24hrs_rep3.Rdata" : "folic-acid",
"FULVESTRANT_timept6hrs_rep1.Rdata" : "fulvestrant",
"FULVESTRANT_timept6hrs_rep2.Rdata" : "fulvestrant",
"FULVESTRANT_timept6hrs_rep3.Rdata" : "fulvestrant",
"FULVESTRANT_timept12hrs_rep1.Rdata" : "fulvestrant",
"FULVESTRANT_timept12hrs_rep2.Rdata" : "fulvestrant",
"FULVESTRANT_timept12hrs_rep3.Rdata" : "fulvestrant",
"FULVESTRANT_timept24hrs_rep1.Rdata" : "fulvestrant",
"FULVESTRANT_timept24hrs_rep2.Rdata" : "fulvestrant",
"FULVESTRANT_timept24hrs_rep3.Rdata" : "fulvestrant",
"GALLAMINE.TRIETHIODIDE_timept6hrs_rep1.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept6hrs_rep2.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept6hrs_rep3.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept12hrs_rep1.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept12hrs_rep2.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept12hrs_rep3.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept24hrs_rep1.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept24hrs_rep2.Rdata" : "gallamine-triethiodide",
"GALLAMINE.TRIETHIODIDE_timept24hrs_rep3.Rdata" : "gallamine-triethiodide",
"Gemcitabine_timept6hrs_rep1.Rdata" : "gemcitabine",
"Gemcitabine_timept6hrs_rep2.Rdata" : "gemcitabine",
"Gemcitabine_timept6hrs_rep3.Rdata" : "gemcitabine",
"Gemcitabine_timept12hrs_rep1.Rdata" : "gemcitabine",
"Gemcitabine_timept12hrs_rep2.Rdata" : "gemcitabine",
"Gemcitabine_timept12hrs_rep3.Rdata" : "gemcitabine",
"Gemcitabine_timept24hrs_rep1.Rdata" : "gemcitabine",
"Gemcitabine_timept24hrs_rep2.Rdata" : "gemcitabine",
"Gemcitabine_timept24hrs_rep3.Rdata" : "gemcitabine",
"GEMFIBROZIL_timept6hrs_rep1.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept6hrs_rep2.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept6hrs_rep3.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept12hrs_rep1.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept12hrs_rep2.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept12hrs_rep3.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept24hrs_rep1.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept24hrs_rep2.Rdata" : "gemfibrozil",
"GEMFIBROZIL_timept24hrs_rep3.Rdata" : "gemfibrozil",
"GLIPIZIDE_timept6hrs_rep1.Rdata" : "glipizide",
"GLIPIZIDE_timept6hrs_rep2.Rdata" : "glipizide",
"GLIPIZIDE_timept6hrs_rep3.Rdata" : "glipizide",
"GLIPIZIDE_timept12hrs_rep1.Rdata" : "glipizide",
"GLIPIZIDE_timept12hrs_rep2.Rdata" : "glipizide",
"GLIPIZIDE_timept12hrs_rep3.Rdata" : "glipizide",
"GLIPIZIDE_timept24hrs_rep1.Rdata" : "glipizide",
"GLIPIZIDE_timept24hrs_rep2.Rdata" : "glipizide",
"GLIPIZIDE_timept24hrs_rep3.Rdata" : "glipizide",
"GRAMICIDIN_timept6hrs_rep1.Rdata" : "gramicidin",
"GRAMICIDIN_timept6hrs_rep2.Rdata" : "gramicidin",
"GRAMICIDIN_timept6hrs_rep3.Rdata" : "gramicidin",
"GRAMICIDIN_timept12hrs_rep1.Rdata" : "gramicidin",
"GRAMICIDIN_timept12hrs_rep2.Rdata" : "gramicidin",
"GRAMICIDIN_timept12hrs_rep3.Rdata" : "gramicidin",
"GRAMICIDIN_timept24hrs_rep1.Rdata" : "gramicidin",
"GRAMICIDIN_timept24hrs_rep2.Rdata" : "gramicidin",
"GRAMICIDIN_timept24hrs_rep3.Rdata" : "gramicidin",
"HEXACHLOROPHENE_timept6hrs_rep1.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept6hrs_rep2.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept6hrs_rep3.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept12hrs_rep1.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept12hrs_rep2.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept12hrs_rep3.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept24hrs_rep1.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept24hrs_rep2.Rdata" : "hexachlorophene",
"HEXACHLOROPHENE_timept24hrs_rep3.Rdata" : "hexachlorophene",
"HYDROCHLOROTHIAZIDE_timept6hrs_rep1.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept6hrs_rep2.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept6hrs_rep3.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept12hrs_rep1.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept12hrs_rep2.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept12hrs_rep3.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept24hrs_rep1.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept24hrs_rep2.Rdata" : "hydrochlorothiazide",
"HYDROCHLOROTHIAZIDE_timept24hrs_rep3.Rdata" : "hydrochlorothiazide",
"IMATINIB_timept6hrs_rep1.Rdata" : "imatinib",
"IMATINIB_timept6hrs_rep2.Rdata" : "imatinib",
"IMATINIB_timept6hrs_rep3.Rdata" : "imatinib",
"IMATINIB_timept12hrs_rep1.Rdata" : "imatinib",
"IMATINIB_timept12hrs_rep2.Rdata" : "imatinib",
"IMATINIB_timept12hrs_rep3.Rdata" : "imatinib",
"IMATINIB_timept24hrs_rep1.Rdata" : "imatinib",
"IMATINIB_timept24hrs_rep2.Rdata" : "imatinib",
"IMATINIB_timept24hrs_rep3.Rdata" : "imatinib",
"INDAPAMIDE_timept6hrs_rep1.Rdata" : "indapamide",
"INDAPAMIDE_timept6hrs_rep2.Rdata" : "indapamide",
"INDAPAMIDE_timept6hrs_rep3.Rdata" : "indapamide",
"INDAPAMIDE_timept12hrs_rep1.Rdata" : "indapamide",
"INDAPAMIDE_timept12hrs_rep2.Rdata" : "indapamide",
"INDAPAMIDE_timept12hrs_rep3.Rdata" : "indapamide",
"INDAPAMIDE_timept24hrs_rep1.Rdata" : "indapamide",
"INDAPAMIDE_timept24hrs_rep2.Rdata" : "indapamide",
"INDAPAMIDE_timept24hrs_rep3.Rdata" : "indapamide",
"INDOMETHACIN_timept6hrs_rep1.Rdata" : "indomethacin",
"INDOMETHACIN_timept6hrs_rep2.Rdata" : "indomethacin",
"INDOMETHACIN_timept6hrs_rep3.Rdata" : "indomethacin",
"INDOMETHACIN_timept12hrs_rep1.Rdata" : "indomethacin",
"INDOMETHACIN_timept12hrs_rep2.Rdata" : "indomethacin",
"INDOMETHACIN_timept12hrs_rep3.Rdata" : "indomethacin",
"INDOMETHACIN_timept24hrs_rep1.Rdata" : "indomethacin",
"INDOMETHACIN_timept24hrs_rep2.Rdata" : "indomethacin",
"INDOMETHACIN_timept24hrs_rep3.Rdata" : "indomethacin",
"Irinotecan.Hydrochloride_timept6hrs_rep1.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept6hrs_rep2.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept6hrs_rep3.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept12hrs_rep1.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept12hrs_rep2.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept12hrs_rep3.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept24hrs_rep1.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept24hrs_rep2.Rdata" : "irinotecan-hydrochloride",
"Irinotecan.Hydrochloride_timept24hrs_rep3.Rdata" : "irinotecan-hydrochloride",
"ISOPROPAMIDE.IODIDE_timept6hrs_rep1.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept6hrs_rep2.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept6hrs_rep3.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept12hrs_rep1.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept12hrs_rep2.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept12hrs_rep3.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept24hrs_rep1.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept24hrs_rep2.Rdata" : "isopropamide-iodide",
"ISOPROPAMIDE.IODIDE_timept24hrs_rep3.Rdata" : "isopropamide-iodide",
"Itraconazole_timept6hrs_rep1.Rdata" : "itraconazole",
"Itraconazole_timept6hrs_rep2.Rdata" : "itraconazole",
"Itraconazole_timept6hrs_rep3.Rdata" : "itraconazole",
"Itraconazole_timept12hrs_rep1.Rdata" : "itraconazole",
"Itraconazole_timept12hrs_rep2.Rdata" : "itraconazole",
"Itraconazole_timept12hrs_rep3.Rdata" : "itraconazole",
"Itraconazole_timept24hrs_rep1.Rdata" : "itraconazole",
"Itraconazole_timept24hrs_rep2.Rdata" : "itraconazole",
"Itraconazole_timept24hrs_rep3.Rdata" : "itraconazole",
"MEBENDAZOLE_timept6hrs_rep1.Rdata" : "mebendazole",
"MEBENDAZOLE_timept6hrs_rep2.Rdata" : "mebendazole",
"MEBENDAZOLE_timept6hrs_rep3.Rdata" : "mebendazole",
"MEBENDAZOLE_timept12hrs_rep1.Rdata" : "mebendazole",
"MEBENDAZOLE_timept12hrs_rep2.Rdata" : "mebendazole",
"MEBENDAZOLE_timept12hrs_rep3.Rdata" : "mebendazole",
"MELPHALAN_timept6hrs_rep1.Rdata" : "melphalan",
"MELPHALAN_timept6hrs_rep2.Rdata" : "melphalan",
"MELPHALAN_timept6hrs_rep3.Rdata" : "melphalan",
"MELPHALAN_timept12hrs_rep1.Rdata" : "melphalan",
"MELPHALAN_timept12hrs_rep2.Rdata" : "melphalan",
"MELPHALAN_timept12hrs_rep3.Rdata" : "melphalan",
"MELPHALAN_timept24hrs_rep1.Rdata" : "melphalan",
"MELPHALAN_timept24hrs_rep2.Rdata" : "melphalan",
"MELPHALAN_timept24hrs_rep3.Rdata" : "melphalan",
"Metixene.hydrochloride_timept6hrs_rep1.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept6hrs_rep2.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept6hrs_rep3.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept12hrs_rep1.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept12hrs_rep2.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept12hrs_rep3.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept24hrs_rep1.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept24hrs_rep2.Rdata" : "metixene-hydrochloride",
"Metixene.hydrochloride_timept24hrs_rep3.Rdata" : "metixene-hydrochloride",
"MICONAZOLE.NITRATE_timept6hrs_rep1.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept6hrs_rep2.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept6hrs_rep3.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept12hrs_rep1.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept12hrs_rep2.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept12hrs_rep3.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept24hrs_rep1.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept24hrs_rep2.Rdata" : "miconazole-nitrate",
"MICONAZOLE.NITRATE_timept24hrs_rep3.Rdata" : "miconazole-nitrate",
"MITOMYCIN.C_timept6hrs_rep1.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept6hrs_rep2.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept6hrs_rep3.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept12hrs_rep1.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept12hrs_rep2.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept12hrs_rep3.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept24hrs_rep1.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept24hrs_rep2.Rdata" : "mitomycin-c",
"MITOMYCIN.C_timept24hrs_rep3.Rdata" : "mitomycin-c",
"MITOXANTHRONE.HYDROCHLORIDE_timept6hrs_rep1.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept6hrs_rep2.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept6hrs_rep3.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept12hrs_rep1.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept12hrs_rep2.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept12hrs_rep3.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept24hrs_rep1.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept24hrs_rep2.Rdata" : "mitoxanthrone-hydrochloride",
"MITOXANTHRONE.HYDROCHLORIDE_timept24hrs_rep3.Rdata" : "mitoxanthrone-hydrochloride",
"Mometasone.furoate_timept6hrs_rep1.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept6hrs_rep2.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept6hrs_rep3.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept12hrs_rep1.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept12hrs_rep2.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept12hrs_rep3.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept24hrs_rep1.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept24hrs_rep2.Rdata" : "mometasone-furoate",
"Mometasone.furoate_timept24hrs_rep3.Rdata" : "mometasone-furoate",
"NISOLDIPINE_timept6hrs_rep1.Rdata" : "nisoldipine",
"NISOLDIPINE_timept6hrs_rep2.Rdata" : "nisoldipine",
"NISOLDIPINE_timept6hrs_rep3.Rdata" : "nisoldipine",
"NISOLDIPINE_timept12hrs_rep1.Rdata" : "nisoldipine",
"NISOLDIPINE_timept12hrs_rep2.Rdata" : "nisoldipine",
"NISOLDIPINE_timept12hrs_rep3.Rdata" : "nisoldipine",
"NISOLDIPINE_timept24hrs_rep1.Rdata" : "nisoldipine",
"NISOLDIPINE_timept24hrs_rep2.Rdata" : "nisoldipine",
"NISOLDIPINE_timept24hrs_rep3.Rdata" : "nisoldipine",
"NITROFURAZONE_timept6hrs_rep1.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept6hrs_rep2.Rdata" : "nitrofurazone",
"NITROFURAZONE_timept6hrs_rep3.Rdata" : "nitrofurazone"}

In [382]:
# GSE102006
dataset_file_pert = {"without_cytokines_Trovafloxacin_6uM_24h.Rdata" : "trovafloxacin",
"with_cytokines_Aspirin_200uM_6h.Rdata" : "aspirin",
"with_cytokines_Aspirin_200uM_12h.Rdata" : "aspirin",
"with_cytokines_Aspirin_200uM_24h.Rdata" : "aspirin",
"with_cytokines_Aspirin_400uM_6h.Rdata" : "aspirin",
"with_cytokines_Aspirin_400uM_12h.Rdata" : "aspirin",
"with_cytokines_Aspirin_400uM_24h.Rdata" : "aspirin",
"with_cytokines_Buspirone_12.5uM_6h.Rdata" : "buspirone",
"with_cytokines_Buspirone_12.5uM_12h.Rdata" : "buspirone",
"with_cytokines_Buspirone_12.5uM_24h.Rdata" : "buspirone",
"with_cytokines_Buspirone_25uM_6h.Rdata" : "buspirone",
"with_cytokines_Buspirone_25uM_12h.Rdata" : "buspirone",
"with_cytokines_Buspirone_25uM_24h.Rdata" : "buspirone",
"with_cytokines_Levofloxacin_3uM_6h.Rdata" : "levofloxacin",
"with_cytokines_Levofloxacin_3uM_12h.Rdata" : "levofloxacin",
"with_cytokines_Levofloxacin_3uM_24h.Rdata" : "levofloxacin",
"with_cytokines_Levofloxacin_6uM_6h.Rdata" : "levofloxacin",
"with_cytokines_Levofloxacin_6uM_12h.Rdata" : "levofloxacin",
"with_cytokines_Levofloxacin_6uM_24h.Rdata" : "levofloxacin",
"with_cytokines_Nefazodone_12.5uM_6h.Rdata" : "nefazodone",
"with_cytokines_Nefazodone_12.5uM_12h.Rdata" : "nefazodone",
"with_cytokines_Nefazodone_12.5uM_24h.Rdata" : "nefazodone",
"with_cytokines_Nefazodone_25uM_6h.Rdata" : "nefazodone",
"with_cytokines_Nefazodone_25uM_12h.Rdata" : "nefazodone",
"with_cytokines_Nefazodone_25uM_24h.Rdata" : "nefazodone",
"with_cytokines_Nimesulide_200uM_6h.Rdata" : "nimesulide",
"with_cytokines_Nimesulide_200uM_12h.Rdata" : "nimesulide",
"with_cytokines_Nimesulide_200uM_24h.Rdata" : "nimesulide",
"with_cytokines_Nimesulide_400uM_6h.Rdata" : "nimesulide",
"with_cytokines_Nimesulide_400uM_12h.Rdata" : "nimesulide",
"with_cytokines_Nimesulide_400uM_24h.Rdata" : "nimesulide",
"with_cytokines_Trovafloxacin_3uM_6h.Rdata" : "trovafloxacin",
"with_cytokines_Trovafloxacin_3uM_12h.Rdata" : "trovafloxacin",
"with_cytokines_Trovafloxacin_3uM_24h.Rdata" : "trovafloxacin",
"with_cytokines_Trovafloxacin_6uM_6h.Rdata" : "trovafloxacin",
"with_cytokines_Trovafloxacin_6uM_12h.Rdata" : "trovafloxacin",
"with_cytokines_Trovafloxacin_6uM_24h.Rdata" : "trovafloxacin",
"without_cytokines_Aspirin_200uM_6h.Rdata" : "aspirin",
"without_cytokines_Aspirin_200uM_12h.Rdata" : "aspirin",
"without_cytokines_Aspirin_200uM_24h.Rdata" : "aspirin",
"without_cytokines_Aspirin_400uM_6h.Rdata" : "aspirin",
"without_cytokines_Aspirin_400uM_12h.Rdata" : "aspirin",
"without_cytokines_Aspirin_400uM_24h.Rdata" : "aspirin",
"without_cytokines_Buspirone_12.5uM_6h.Rdata" : "buspirone",
"without_cytokines_Buspirone_12.5uM_12h.Rdata" : "buspirone",
"without_cytokines_Buspirone_12.5uM_24h.Rdata" : "buspirone",
"without_cytokines_Buspirone_25uM_6h.Rdata" : "buspirone",
"without_cytokines_Buspirone_25uM_12h.Rdata" : "buspirone",
"without_cytokines_Buspirone_25uM_24h.Rdata" : "buspirone",
"without_cytokines_Levofloxacin_3uM_6h.Rdata" : "levofloxacin",
"without_cytokines_Levofloxacin_3uM_12h.Rdata" : "levofloxacin",
"without_cytokines_Levofloxacin_3uM_24h.Rdata" : "levofloxacin",
"without_cytokines_Levofloxacin_6uM_6h.Rdata" : "levofloxacin",
"without_cytokines_Levofloxacin_6uM_12h.Rdata" : "levofloxacin",
"without_cytokines_Levofloxacin_6uM_24h.Rdata" : "levofloxacin",
"without_cytokines_Nefazodone_12.5uM_6h.Rdata" : "nefazodone",
"without_cytokines_Nefazodone_12.5uM_12h.Rdata" : "nefazodone",
"without_cytokines_Nefazodone_12.5uM_24h.Rdata" : "nefazodone",
"without_cytokines_Nefazodone_25uM_6h.Rdata" : "nefazodone",
"without_cytokines_Nefazodone_25uM_12h.Rdata" : "nefazodone",
"without_cytokines_Nefazodone_25uM_24h.Rdata" : "nefazodone",
"without_cytokines_Nimesulide_200uM_6h.Rdata" : "nimesulide",
"without_cytokines_Nimesulide_200uM_12h.Rdata" : "nimesulide",
"without_cytokines_Nimesulide_200uM_24h.Rdata" : "nimesulide",
"without_cytokines_Nimesulide_400uM_6h.Rdata" : "nimesulide",
"without_cytokines_Nimesulide_400uM_12h.Rdata" : "nimesulide",
"without_cytokines_Nimesulide_400uM_24h.Rdata" : "nimesulide",
"without_cytokines_Trovafloxacin_3uM_6h.Rdata" : "trovafloxacin",
"without_cytokines_Trovafloxacin_3uM_12h.Rdata" : "trovafloxacin",
"without_cytokines_Trovafloxacin_3uM_24h.Rdata" : "trovafloxacin",
"without_cytokines_Trovafloxacin_6uM_6h.Rdata" : "trovafloxacin",
"without_cytokines_Trovafloxacin_6uM_12h.Rdata" : "trovafloxacin"}

In [390]:
# GSE51068
dataset_file_pert = {"Rapamycin_22uM_24hrs.Rdata" : "rapamycin",
"TrichostatinA_0.114uM_6hrs.Rdata" : "trichostatin-a",
"TrichostatinA_0.114uM_12hrs.Rdata" : "trichostatin-a",
"TrichostatinA_0.114uM_24hrs.Rdata" : "trichostatin-a",
"TrichostatinA_0.143uM_6hrs.Rdata" : "trichostatin-a",
"TrichostatinA_0.143uM_12hrs.Rdata" : "trichostatin-a",
"TrichostatinA_0.143uM_24hrs.Rdata" : "trichostatin-a",
"Vincristine_0.01uM_6hrs.Rdata" : "vincristine",
"Vincristine_0.01uM_24hrs.Rdata" : "vincristine",
"Vincristine_0.005uM_6hrs.Rdata" : "vincristine",
"Blebbistatin_10uM_6hrs.Rdata" : "blebbistatin",
"Blebbistatin_10uM_12hrs.Rdata" : "blebbistatin",
"Blebbistatin_100uM_6hrs.Rdata" : "blebbistatin",
"Blebbistatin_100uM_12hrs.Rdata" : "blebbistatin",
"Blebbistatin_100uM_24hrs.Rdata" : "blebbistatin",
"Camptothecin_0.01uM_6hrs.Rdata" : "camptothecin",
"Camptothecin_0.01uM_12hrs.Rdata" : "camptothecin",
"Camptothecin_0.01uM_24hrs.Rdata" : "camptothecin",
"Camptothecin_0.31uM_6hrs.Rdata" : "camptothecin",
"Camptothecin_0.31uM_12hrs.Rdata" : "camptothecin",
"Camptothecin_0.31uM_24hrs.Rdata" : "camptothecin",
"Cycloheximide_0.264uM_6hrs.Rdata" : "cycloheximide",
"Cycloheximide_0.264uM_24hrs.Rdata" : "cycloheximide",
"Cycloheximide_5uM_6hrs.Rdata" : "cycloheximide",
"Cycloheximide_5uM_12hrs.Rdata" : "cycloheximide",
"Cycloheximide_5uM_24hrs.Rdata" : "cycloheximide",
"Doxorubicinhydrochloride_0.03uM_6hrs.Rdata" : "doxorubicinhydrochloride",
"Doxorubicinhydrochloride_0.03uM_12hrs.Rdata" : "doxorubicinhydrochloride",
"Doxorubicinhydrochloride_0.03uM_24hrs.Rdata" : "doxorubicinhydrochloride",
"Doxorubicinhydrochloride_0.101uM_6hrs.Rdata" : "doxorubicinhydrochloride",
"Doxorubicinhydrochloride_0.101uM_12hrs.Rdata" : "doxorubicinhydrochloride",
"Doxorubicinhydrochloride_0.101uM_24hrs.Rdata" : "doxorubicinhydrochloride",
"Etoposide_0.811uM_6hrs.Rdata" : "etoposide",
"Etoposide_0.811uM_12hrs.Rdata" : "etoposide",
"Etoposide_0.811uM_24hrs.Rdata" : "etoposide",
"Etoposide_0.812uM_6hrs.Rdata" : "etoposide",
"Etoposide_0.812uM_12hrs.Rdata" : "etoposide",
"Etoposide_0.812uM_24hrs.Rdata" : "etoposide",
"Geldanamycin_0.032uM_6hrs.Rdata" : "geldanamycin",
"Geldanamycin_0.032uM_12hrs.Rdata" : "geldanamycin",
"Geldanamycin_0.032uM_24hrs.Rdata" : "geldanamycin",
"H-7_Dihydrochloride_12.4uM_6hrs.Rdata" : "h-7",
"H-7_Dihydrochloride_12.4uM_12hrs.Rdata" : "h-7",
"H-7_Dihydrochloride_12.4uM_24hrs.Rdata" : "h-7",
"H-7_Dihydrochloride_20.2uM_6hrs.Rdata" : "h-7",
"H-7_Dihydrochloride_20.2uM_12hrs.Rdata" : "h-7",
"H-7_Dihydrochloride_20.2uM_24hrs.Rdata" : "h-7",
"Methotrexate_100uM_24hrs.Rdata" : "methotrexate",
"MitomycinC_0.553uM_6hrs.Rdata" : "mitomycin-c",
"MitomycinC_0.553uM_24hrs.Rdata" : "mitomycin-c",
"MitomycinC_2.56uM_6hrs.Rdata" : "mitomycin-c",
"MitomycinC_2.56uM_12hrs.Rdata" : "mitomycin-c",
"MitomycinC_2.56uM_24hrs.Rdata" : "mitomycin-c",
"Monastrol_50uM_6hrs.Rdata" : "monastrol",
"Monastrol_50uM_24hrs.Rdata" : "monastrol",
"Monastrol_100uM_6hrs.Rdata" : "monastrol",
"Monastrol_100uM_12hrs.Rdata" : "monastrol",
"Monastrol_100uM_24hrs.Rdata" : "monastrol",
"Rapamycin_13.8uM_6hrs.Rdata" : "rapamycin",
"Rapamycin_13.8uM_12hrs.Rdata" : "rapamycin",
"Rapamycin_13.8uM_24hrs.Rdata" : "rapamycin",
"Rapamycin_22uM_6hrs.Rdata" : "rapamycin",
"Rapamycin_22uM_12hrs.Rdata" : "rapamycin"}

In [391]:
dataset = 'GSE51068'
ilincs_ids = pd.read_csv(f"{dataset}_pert_ids.csv", sep = '\t')
ilincs_ids = ilincs_ids.set_index('pert')


## findConcordancesSC

In [392]:
min_procentiles_findConcordancesSC = {}
position_in_list_findConcordancesSC = {}
list_len_findConcordancesSC = {}

for subdir, dirs, files in os.walk(f"sigs_to_test//{dataset}_ilincs//results_findConcordancesSC"):
    for file in files:
        if os.stat(f"sigs_to_test//{dataset}_ilincs//results_findConcordancesSC//{file}").st_size <5:
            continue
        df = pd.read_csv(f"sigs_to_test//{dataset}_ilincs//results_findConcordancesSC//{file}",sep = '\t')
        if 'lincsPertID' not in df.columns:
            continue
        name = file[:-len("_ilincs.csv_ilincs_results.csv")]
        if name not in min_procentiles_findConcordancesSC:
            min_procentiles_findConcordancesSC[name] = []
            position_in_list_findConcordancesSC[name] = []
            list_len_findConcordancesSC[name] = []
            
        pert_from_experiment = dataset_file_pert[name]
        ilincs_id_list = ilincs_ids.loc[pert_from_experiment]['iLincs id']
        if pd.isna(ilincs_id_list):
            continue
        ilincs_from_experiment = ilincs_id_list.split(',')
        
        for ilincs_id in ilincs_from_experiment:
            df_find = df['lincsPertID'].str.find(ilincs_id)
            indexes = df_find[df_find==0].index.tolist()
            if not indexes:
                continue
            min_position_pert = min(indexes)
            
            position_in_list = min_position_pert
            list_len = df_find.shape[0]
            min_procentile_pert = (1+position_in_list)/list_len
            min_procentiles_findConcordancesSC[name].append(min_procentile_pert)
            position_in_list_findConcordancesSC[name].append(position_in_list)
            list_len_findConcordancesSC[name].append(list_len)

In [393]:
for pert in min_procentiles_findConcordancesSC:
    if min_procentiles_findConcordancesSC[pert]:
        min_proc = min(min_procentiles_findConcordancesSC[pert])
        min_proc_index = min_procentiles_findConcordancesSC[pert].index(min_proc)
        min_procentiles_findConcordancesSC[pert] = min_proc
        position_in_list_findConcordancesSC[pert] = position_in_list_findConcordancesSC[pert][min_proc_index]
        list_len_findConcordancesSC[pert] = list_len_findConcordancesSC[pert][min_proc_index]
    else:
        min_procentiles_findConcordancesSC[pert] = None
        position_in_list_findConcordancesSC[pert] = None
        list_len_findConcordancesSC[pert] = None
        
pd.DataFrame({'sample' : list(min_procentiles_findConcordancesSC.keys()),
 'min_pert' : list(min_procentiles_findConcordancesSC.values()),
 'min_position' : list(position_in_list_findConcordancesSC.values()),
 'result_len' : list(list_len_findConcordancesSC.values())}).to_csv(f'results//{dataset}_findConcordancesSC.csv', 
                                                                         sep = '\t')

# Checking L1000FWD ranks


In [394]:
min_procentiles_L1000FWD = {}
position_in_list_L1000FWD = {}
list_len_L1000FWD = {}

for subdir, dirs, files in os.walk(f"sigs_to_test//{dataset}_L1000FWD//"):
    for file in files:
        if os.stat(f"sigs_to_test//{dataset}_L1000FWD//{file}").st_size <5:
            continue
        df = pd.read_csv(f"sigs_to_test//{dataset}_L1000FWD//{file}",sep = '\t')
        if 'pert_id' not in df.columns:
            continue
        name = file[:-len("_L1000FWD.csv")]
        
        if name not in min_procentiles_L1000FWD:
            min_procentiles_L1000FWD[name] = []
            position_in_list_L1000FWD[name] = []
            list_len_L1000FWD[name] = []
            
        pert_from_experiment = dataset_file_pert[name]
        
        ilincs_id_list = ilincs_ids.loc[pert_from_experiment]['pert id']
        if pd.isna(ilincs_id_list):
            continue
        ilincs_from_experiment = ilincs_id_list.split(',')
        
        for ilincs_id in ilincs_from_experiment:
            df_find = df['pert_id'].str.find(ilincs_id)
            indexes = df_find[df_find==0].index.tolist()
            if not indexes:
                continue
            min_position_pert = min(indexes)
            position_in_list = min_position_pert
            list_len = df_find.shape[0]
            min_procentile_pert = (1 + position_in_list)/list_len
            min_procentiles_L1000FWD[name].append(min_procentile_pert)
            position_in_list_L1000FWD[name].append(position_in_list)
            list_len_L1000FWD[name].append(list_len)

In [395]:
for pert in min_procentiles_L1000FWD:
    if min_procentiles_L1000FWD[pert]:
        min_proc = min(min_procentiles_L1000FWD[pert])
        min_proc_index = min_procentiles_L1000FWD[pert].index(min_proc)
        min_procentiles_L1000FWD[pert] = min_proc
        position_in_list_L1000FWD[pert] = position_in_list_L1000FWD[pert][min_proc_index]
        list_len_L1000FWD[pert] = list_len_L1000FWD[pert][min_proc_index]
        
    else:
        min_procentiles_L1000FWD[pert] = None
        position_in_list_L1000FWD[pert] = None
        list_len_L1000FWD[pert] = None
        
pd.DataFrame({'sample' : list(min_procentiles_L1000FWD.keys()),
 'min_pert' : list(min_procentiles_L1000FWD.values()),
 'min_position' : list(position_in_list_L1000FWD.values()),
 'result_len' : list(list_len_L1000FWD.values())}).to_csv(f'results//{dataset}_L1000FWD.csv', 
                                                                         sep = '\t')

# Checking CLUE ranks


In [396]:
GSE60408_dict = {'Cladribine_timept6hrs_rep1' : '62c312853b536d0012dbf0f2',
'Butoconazole.nitrate_timept6hrs_rep1' : '62c31285fc243200123b5bc8',
'DYCLONINE.HYDROCHLORIDE_timept6hrs_rep1' : '62c312843b536d0012dbf0f0',
'CLOZAPINE_timept6hrs_rep1' : '62c31282fc243200123b5bc6',
'ACRISORCIN_timept6hrs_rep1' : '62c312813b536d0012dbf0ee',
'FULVESTRANT_timept6hrs_rep1' : '62c31280fc243200123b5bc4',
'MITOMYCIN.C_timept6hrs_rep1' : '62c3127dfc243200123b5bc2',
'ENOXACIN_timept6hrs_rep1' : '62c3127b3b536d0012dbf0ec',
'GALLAMINE.TRIETHIODIDE_timept6hrs_rep1' : '62c3035e3b536d0012dbf0ea',
'MELPHALAN_timept6hrs_rep1' : '62c3035cfc243200123b5bbc',
'Auranofin_timept6hrs_rep1' : '62c3035a3b536d0012dbf0e8',
'FELODIPINE_timept6hrs_rep1' : '62c30356fc243200123b5bba',
'ERGOCALCIFEROL_timept6hrs_rep1' : '62c303503b536d0012dbf0e6',
'AMIODARONE.HYDROCHLORIDE_timept6hrs_rep1' : '62c3034ffc243200123b5bb8',
'Gemcitabine_timept6hrs_rep1' : '62c3034e3b536d0012dbf0e4',
'ETHOPROPAZINE.HYDROCHLORIDE_timept6hrs_rep1' : '62c3034cfc243200123b5bb6',
'GLIPIZIDE_timept6hrs_rep1' : '62c2fcf73b536d0012dbf0e0',
'Irinotecan.Hydrochloride_timept6hrs_rep1' : '62c2fcf6fc243200123b5bb1',
'ALTRETAMINE_timept6hrs_rep1' : '62c2fcf13b536d0012dbf0de',
'GRAMICIDIN_timept6hrs_rep1' : '62c2fcf0fc243200123b5baf',
'Docetaxel_timept6hrs_rep1' : '62c2fce8fc243200123b5bad',
'DIMERCAPROL_timept6hrs_rep1' : '62c2fce63b536d0012dbf0dc',
'NITROFURAZONE_timept6hrs_rep1' : '62c2fce0fc243200123b5bab',
'MITOXANTHRONE.HYDROCHLORIDE_timept6hrs_rep1' : '62c2fcdf3b536d0012dbf0da',
'DIGITOXIN_timept6hrs_rep1' : '62c2fcdffc243200123b5ba9',
'CYTARABINE_timept6hrs_rep1' : '62c2fcd93b536d0012dbf0d8'}


GSE51068_dict = {'MitomycinC_2.56uM_6hrs' : '62cd4cccfc243200123b5ffd',
'Cycloheximide_0.264uM_24hrs' : '62cd4ccc3b536d0012dbf521',
'Geldanamycin_0.032uM_12hrs' : '62cd4ccefc243200123b5fff',
'Monastrol_50uM_6hrs' : '62cd4cce3b536d0012dbf523',
'Geldanamycin_0.032uM_24hrs' : '62cd4cd0fc243200123b6001',
'Blebbistatin_100uM_12hrs' : '62cd4cd03b536d0012dbf525',
'Blebbistatin_100uM_6hrs' : '62cd4cd1fc243200123b6003',
'H-7_Dihydrochloride_12.4uM_12hrs' : '62cd4cd23b536d0012dbf527',
'Vincristine_0.01uM_24hrs' : '62cd4cd2fc243200123b6005'}

GSE102006_dict = {'with_cytokines_Nimesulide_200uM_12h' : '6291a2e038eca60012254376',
'with_cytokines_Nimesulide_200uM_24h' : '6291a2e238eca60012254378',
'without_cytokines_Aspirin_400uM_6h' : '6291a2e1fcaa4f00126683e6',
'without_cytokines_Trovafloxacin_6uM_12h' : '6291a2e0fcaa4f00126683e4'}

current_dict = GSE51068_dict
min_procentiles_clue = {}
position_in_list_clue = {}
list_len_clue = {}


for ex in current_dict:
    rdata_object = ex+'.Rdata'
    name = rdata_object
    if name not in min_procentiles_clue:
        min_procentiles_clue[name] = []
        position_in_list_clue[name] = []
        list_len_clue[name] = []
            
    job_id = current_dict[ex]
    pert_from_experiment=dataset_file_pert[rdata_object]
           
    ilincs_id_list = ilincs_ids.loc[pert_from_experiment]['pert id']
    if pd.isna(ilincs_id_list):
        continue
        
    ilincs_from_experiment = ilincs_id_list.split(',')
    for ilincs_id in ilincs_from_experiment:
        res = parse(f"sigs_to_test//{dataset}_clue//my_analysis.sig_gutc_tool.{job_id}//matrices//gutc//ps_pert_summary.gctx")
        res = res.data_df.sort_values(by=['TAG'], ascending=False)       
        if ilincs_id not in list(res.index):
            continue
        
        position_in_list = list(res.index).index(ilincs_id)
        list_len = res.shape[0]      
        min_procentile_pert = (1+position_in_list)/list_len
        min_procentiles_clue[name].append(min_procentile_pert)
        position_in_list_clue[name].append(position_in_list)
        list_len_clue[name].append(list_len)
            


In [397]:
for pert in min_procentiles_clue:
    if min_procentiles_clue[pert]:
        min_proc = min(min_procentiles_clue[pert])
        min_proc_index = min_procentiles_clue[pert].index(min_proc)
        min_procentiles_clue[pert] = min_proc
        position_in_list_clue[pert] = position_in_list_clue[pert][min_proc_index]
        list_len_clue[pert] = list_len_clue[pert][min_proc_index]
        
    else:
        min_procentiles_clue[pert] = None
        position_in_list_clue[pert] = None
        list_len_clue[pert] = None
        
pd.DataFrame({'sample' : list(min_procentiles_clue.keys()),
 'min_pert' : list(min_procentiles_clue.values()),
 'min_position' : list(position_in_list_clue.values()),
 'result_len' : list(list_len_clue.values())}).to_csv(f'results//{dataset}_clue.csv', 
                                                                         sep = '\t')
